In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['dataset_TSMC2014_TKY.csv', 'dataset_TSMC2014_NYC.csv']


In [2]:
ny_data = pd.read_csv("../input/dataset_TSMC2014_NYC.csv")
ny_data.head()

,userId,venueId,venueCategoryId,venueCategory,latitude,longitude,timezoneOffset,utcTimestamp
0,470,49bbd6c0f964a520f4531fe3,4bf58dd8d48988d127951735,Arts & Crafts Store,40.719810,-74.002581,-240,Tue Apr 03 18:00:09 +0000 2012
1,979,4a43c0aef964a520c6a61fe3,4bf58dd8d48988d1df941735,Bridge,40.606800,-74.044170,-240,Tue Apr 03 18:00:25 +0000 2012
2,69,4c5cc7b485a1e21e00d35711,4bf58dd8d48988d103941735,Home (private),40.716162,-73.883070,-240,Tue Apr 03 18:02:24 +0000 2012
3,395,4bc7086715a7ef3bef9878da,4bf58dd8d48988d104941735,Medical Center,40.745164,-73.982519,-240,Tue Apr 03 18:02:41 +0000 2012
4,87,4cf2c5321d18a143951b5cec,4bf58dd8d48988d1cb941735,Food Truck,40.740104,-73.989658,-240,Tue Apr 03 18:03:00 +0000 2012


In [3]:
ny_data.describe()

,userId,latitude,longitude,timezoneOffset
count,227428.000000,227428.000000,227428.000000,227428.000000
mean,535.304817,40.754045,-73.974556,-253.392019
std,305.398774,0.072458,0.086209,43.234750
min,1.000000,40.550852,-74.274766,-420.000000
25%,287.000000,40.718330,-74.000633,-240.000000
50%,523.000000,40.747745,-73.983479,-240.000000
75%,793.000000,40.778374,-73.945709,-240.000000
max,1083.000000,40.988332,-73.683825,660.000000


In [4]:
import datetime
data = []
for utcoffset_index, utcoffset in enumerate(ny_data['utcTimestamp']):    
    year = datetime.datetime.strptime(utcoffset, '%a %b %d %X %z %Y').strftime('%Y')
    month = datetime.datetime.strptime(utcoffset, '%a %b %d %X %z %Y').strftime('%m')
    day = datetime.datetime.strptime(utcoffset, '%a %b %d %X %z %Y').strftime('%d')
    weekday = datetime.datetime.strptime(utcoffset, '%a %b %d %X %z %Y').strftime('%a')
    time = datetime.datetime.strptime(utcoffset, '%a %b %d %X %z %Y').strftime('%X')
    data.append([year, month, day, weekday, time])
ny_data_fe = pd.DataFrame(data, columns = ['year', 'month', 'day', 'weekday', 'time'])

for col in ny_data_fe.columns:
    if col not in ['weekday', 'time']:
        ny_data[col] = pd.to_numeric(ny_data_fe[col])
    else:
        ny_data[col] = ny_data_fe[col]

In [5]:
ny_data.describe()

,userId,latitude,longitude,timezoneOffset,year,month,day
count,227428.000000,227428.000000,227428.000000,227428.000000,227428.000000,227428.000000,227428.000000
mean,535.304817,40.754045,-73.974556,-253.392019,2012.101034,6.249512,14.683794
std,305.398774,0.072458,0.086209,43.234750,0.301375,3.023309,8.151403
min,1.000000,40.550852,-74.274766,-420.000000,2012.000000,1.000000,1.000000
25%,287.000000,40.718330,-74.000633,-240.000000,2012.000000,4.000000,8.000000
50%,523.000000,40.747745,-73.983479,-240.000000,2012.000000,5.000000,14.000000
75%,793.000000,40.778374,-73.945709,-240.000000,2012.000000,8.000000,21.000000
max,1083.000000,40.988332,-73.683825,660.000000,2013.000000,12.000000,31.000000


In [6]:
# ny_data.drop('utcTimestamp', 1, inplace=True)
ny_data['year'] = np.where(ny_data['year'] == 2012, 1, 0)
ny_data.head()

,userId,venueId,venueCategoryId,venueCategory,latitude,longitude,timezoneOffset,utcTimestamp,year,month,day,weekday,time
0,470,49bbd6c0f964a520f4531fe3,4bf58dd8d48988d127951735,Arts & Crafts Store,40.719810,-74.002581,-240,Tue Apr 03 18:00:09 +0000 2012,1,4,3,Tue,18:00:09
1,979,4a43c0aef964a520c6a61fe3,4bf58dd8d48988d1df941735,Bridge,40.606800,-74.044170,-240,Tue Apr 03 18:00:25 +0000 2012,1,4,3,Tue,18:00:25
2,69,4c5cc7b485a1e21e00d35711,4bf58dd8d48988d103941735,Home (private),40.716162,-73.883070,-240,Tue Apr 03 18:02:24 +0000 2012,1,4,3,Tue,18:02:24
3,395,4bc7086715a7ef3bef9878da,4bf58dd8d48988d104941735,Medical Center,40.745164,-73.982519,-240,Tue Apr 03 18:02:41 +0000 2012,1,4,3,Tue,18:02:41
4,87,4cf2c5321d18a143951b5cec,4bf58dd8d48988d1cb941735,Food Truck,40.740104,-73.989658,-240,Tue Apr 03 18:03:00 +0000 2012,1,4,3,Tue,18:03:00


In [7]:
grouped = ny_data.groupby(by='venueCategoryId')

for _, group in grouped:
    print(group.head())

      userId                   venueId    ...    weekday      time
323       47  4a914773f964a520dd1920e3    ...        Tue  21:29:21
377      381  4e405e39483b04e17abb677f    ...        Tue  21:51:16
403      637  4a914773f964a520dd1920e3    ...        Tue  21:57:54
1250      84  4ee73945e30005f8ba684170    ...        Wed  10:16:04
2046     381  4e405e39483b04e17abb677f    ...        Wed  17:43:23

[5 rows x 13 columns]
      userId                   venueId    ...    weekday      time
713      824  4e2d913514955dbf7aefc95a    ...        Wed  00:04:09
1954     389  4a902d50f964a5205c1620e3    ...        Wed  17:00:08
2128    1027  4b4bc857f964a52085a726e3    ...        Wed  18:23:06
2301     374  4d98ca65b188721ea17d3037    ...        Wed  21:31:30
2365     328  4afb40d6f964a520661c22e3    ...        Wed  22:54:35

[5 rows x 13 columns]
      userId                   venueId    ...    weekday      time
271      557  4ae23a21f964a5200a8c21e3    ...        Tue  21:00:07
725      529  4c

In [8]:
selected_features = ['latitude', 'longitude', 'timezoneOffset']

In [9]:
data, labels = ny_data[selected_features], ny_data['venueCategory']
data.shape

(227428, 3)

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.30, random_state=42)

X_train.shape, X_test.shape, y_test.shape, y_train.shape

((159199, 3), (68229, 3), (68229,), (159199,))

In [11]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

clf.fit(X_train, y_train)
clf.score(X_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9904836085653804

In [12]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

predictions = clf.predict(X_test)

print(classification_report(y_test, predictions))

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                                          precision    recall  f1-score   support

                       Afghan Restaurant       0.00      0.00      0.00         2
                      African Restaurant       0.69      0.69      0.69        13
                                 Airport       0.95      0.98      0.96       859
                     American Restaurant       0.82      0.82      0.82      1138
                          Animal Shelter       0.84      0.89      0.86        18
                            Antique Shop       0.60      0.60      0.60         5
                                Aquarium       1.00      1.00      1.00         1
                                  Arcade       0.88      0.94      0.91       114
                        Arepa Restaurant       0.56      0.83      0.67         6
                  Argentinian Restaurant       1.00      0.75      0.86         4
                             Art Gallery       0.68      0.59      0.63       151
               

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [13]:
clf.score(X_test, y_test)

0.8633132538949714